# Exemplo de Integração de Dados


### Funcionalidades demonstradas

1. Mapeamento dos esquemas dos arquivos CSVs presentes no diretório data/input/users através da correlação das colunas correspondentes (seguindo o arquivo schema_mapping.json presente na pasta config);

2. Conversão do formato dos arquivo integrado para um formato colunar de alta performance de leitura (Parquet).

3. Deduplicação dos dados convertidos: No conjunto de dados convertidos há múltiplas entradas para um mesmo registro, variando apenas os valores de alguns dos campos entre elas. Foi necessário realizar um processo de deduplicação destes dados, a fim de apenas manter a última entrada de cada registro, usando como referência o id para identificação dos registros duplicados e a data de atualização (update_date) para definição do registro mais recente;

4. Conversão do tipo dos dados deduplicados: No diretório config há um arquivo JSON de configuração (types_mapping.json), contendo os nomes dos campos e os respectivos tipos desejados de output. Utilizando esse arquivo como input, foi realizado um processo de conversão dos tipos dos campos descritos, no conjunto de dados deduplicados.

### Notas gerais
- Todas as operações foram realizadas utilizando Spark.

- Cada operação utilizou como base o dataframe resultante do passo anterior, sendo persistido em arquivos Parquet.

- Houve a transformação de tipos de dados em alguns campos (id, age, create_date, update_date)

### Referências

[1] PLASE, D.; NIEDRITE, L.; TARANOVS, R. A comparison of HDFS compact data formats: Avro versus Parquet / HDFS glaustųjų duomenų formatų palyginimas: Avro prieš Parquet. Mokslas – Lietuvos ateitis / Science – Future of Lithuania, v. 9, n. 3, p. 267-276, 4 jul. 2017.

## Carregamento dos arquivos de entrada e configuração

In [3]:
!wget -P data/input/users/ https://raw.githubusercontent.com/ifpb/Integracao-dados-overview/main/data/input/users/load.csv
!wget -P data/input/users/ https://raw.githubusercontent.com/ifpb/Integracao-dados-overview/main/data/input/users/load2.csv
!wget -P config/ https://raw.githubusercontent.com/ifpb/Integracao-dados-overview/main/config/schema_mapping.json
!wget -P config/ https://raw.githubusercontent.com/ifpb/Integracao-dados-overview/main/config/types_mapping_full.json

--2023-08-25 10:37:11--  https://raw.githubusercontent.com/ifpb/Integracao-dados-overview/main/data/input/users/load.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655 [text/plain]
Saving to: 'data/input/users/load.csv.2'

load.csv.2          100%[===================>]     655  --.-KB/s    in 0s      

2023-08-25 10:37:12 (27.2 MB/s) - 'data/input/users/load.csv.2' saved [655/655]

--2023-08-25 10:37:12--  https://raw.githubusercontent.com/ifpb/Integracao-dados-overview/main/data/input/users/load2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.

## Instalação das dependências

In [4]:
!pip install findspark==2.0.1
!pip install numpy==1.21.6
!pip install pandas==1.3.5
!pip install py4j==0.10.9.5
!pip install pyspark==3.3.0
!pip install python-dateutil==2.8.2
!pip install pytz==2022.2.1
!pip install six==1.16.0

## Carregamento da Configuração


In [5]:
CONFIG = {
    'INPUT_FILE_1': 'data/input/users/load.csv',
    'INPUT_FILE_2': 'data/input/users/load2.csv',
    'TYPES_MAPPING': 'config/types_mapping_full.json',
    'SCHEMA_MAPPING': 'config/schema_mapping.json',
    'OUTPUT_PATH': 'data/output/users.parquet',
    'OUTPUT_PATH_CSV': 'data/output/users.csv',
    'OUTPUT_PATH_DEDUPLICATED': 'data/output/users-deduplicated.parquet',
    'APP_NAME': 'Demonstracao-IntegracaoDados'
}

## Inicialização do Spark

In [6]:
import os

import findspark
from pyspark.sql.types import StructType

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName(CONFIG['APP_NAME']).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/25 10:37:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Etapa 1 - Mapeamento de Esquemas e conversão de formatos

In [17]:

"""
Carrega o arquivo de entrada (load.csv), realiza mapeamento do esquema conforme o arquivo JSON (ex., types_mapping.json)

@see config.py para ter acesso às configurações dos arquivos que são carregados 
"""
def convert_data_types_and_formats():

    ## De posse do esquema pronto a ser utilizado, é feito o carregamento dos dados do CSV
    df1 = spark.read.csv(CONFIG['INPUT_FILE_1'], header=True, mode="DROPMALFORMED")
    df2 = spark.read.csv(CONFIG['INPUT_FILE_2'], header=True, inferSchema=True)
    df2 = df2.drop("internal_id")

    df1.show()
    df2.show()

    df1.createOrReplaceTempView('df1')
    df2.createOrReplaceTempView('df2')

    df3 = spark.sql('SELECT email, name, phone, address, age, create_date, update_date FROM df1'
                    ' UNION ALL '
                    'SELECT email_address, fullname, telephone, address, age, create_date, update_date FROM df2')

    df3.show()

    ## Salva os dados carregados como Parquet no diretório indicado
    if not os.path.isdir(CONFIG['OUTPUT_PATH']):
        df3.write.parquet(CONFIG['OUTPUT_PATH'])

convert_data_types_and_formats()


+---+--------------------+---------------+---------------+--------------------+---+--------------------+--------------------+
| id|               email|           name|          phone|             address|age|         create_date|         update_date|
+---+--------------------+---------------+---------------+--------------------+---+--------------------+--------------------+
|  1|david.lynch@compa...|    David Lynch|(11) 99999-9997|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-03-03 18:47:...|
|  1|david.lynch@compa...|    David Lynch|(11) 99999-9998|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-04-14 17:09:...|
|  2|sherlock.holmes@c...|Sherlock Holmes|(11) 94815-1623|221B Baker Street...| 34|2018-04-21 20:21:...|2018-04-21 20:21:...|
|  1|david.lynch@compa...|    David Lynch|(11) 99999-9999|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-05-23 10:13:...|
+---+--------------------+---------------+---------------+--------------------+---+--------------------+--------------

## Etapa 2 - Deduplicação de Dados

In [32]:
import pyspark.sql.functions as func

"""
Recupera a lista de usuários construída no arquivo (converter.py) e realiza a remoção das instâncias duplicadas
"""
def deduplicate():
    ## Carrega lista de usuários persistida no formato Parquet
    users = spark.read.parquet(CONFIG['OUTPUT_PATH'])

    users.createOrReplaceTempView('users')

    users_deduplicated = spark.sql('SELECT id, max(update_date), max(email) as email, max(name) as name, max(phone) as phone, max(age) as age, max(create_date) as create_date FROM users GROUP BY id');

    users_deduplicated.show()

    ## Persiste o novo dataframe em um novo Parquet
    if not os.path.isdir(CONFIG['OUTPUT_PATH_DEDUPLICATED']):
        users_deduplicated.write.parquet(CONFIG['OUTPUT_PATH_DEDUPLICATED'])
    return users_deduplicated

deduplicate()

+---+--------------------+--------------------+--------------------+---------------+---+--------------------+
| id|    max(update_date)|               email|                name|          phone|age|         create_date|
+---+--------------------+--------------------+--------------------+---------------+---+--------------------+
|  1|2018-05-23 10:13:...|david.lynch@compa...|         David Lynch|(11) 99999-9999| 72|2018-03-03 18:47:...|
|  2|2018-04-21 20:21:...|sherlock.holmes@c...|     Sherlock Holmes|(11) 94815-1623| 34|2018-04-21 20:21:...|
|  3|2018-05-19 05:08:...|spongebob.squarep...|Spongebob Squarep...|(11) 98765-4321| 13|2018-05-19 04:07:...|
+---+--------------------+--------------------+--------------------+---------------+---+--------------------+



DataFrame[id: int, max(update_date): timestamp, email: string, name: string, phone: string, age: int, create_date: timestamp]

## Etapa 3 - Operações

In [33]:
def operations():
    users = spark.read.parquet(CONFIG['OUTPUT_PATH_DEDUPLICATED'])
    users = users.sort(users.id.asc())
    users.show()
    print("Total de usuários = ", users.count())
    users_pd = users.toPandas()
    print("Média de idade = ", users_pd['age'].mean())
    print("Usuário mais velho = ", users.select('name', 'email', 'age').sort(users.age.desc()).first())
    print("Usuário mais novo = ", users.select('name', 'email', 'age').sort(users.age.asc()).first())

operations()

+---+--------------------+--------------------+--------------------+---------------+--------------------+---+--------------------+
| id|         update_date|               email|                name|          phone|             address|age|         create_date|
+---+--------------------+--------------------+--------------------+---------------+--------------------+---+--------------------+
|  1|2018-05-23 10:13:...|david.lynch@compa...|         David Lynch|(11) 99999-9999|Mulholland Drive,...| 72|2018-03-03 18:47:...|
|  2|2018-04-21 20:21:...|sherlock.holmes@c...|     Sherlock Holmes|(11) 94815-1623|221B Baker Street...| 34|2018-04-21 20:21:...|
|  3|2018-05-19 05:08:...|spongebob.squarep...|Spongebob Squarep...|(11) 98765-4321|122 Conch Street,...| 13|2018-05-19 04:07:...|
+---+--------------------+--------------------+--------------------+---------------+--------------------+---+--------------------+

Total de usuários =  3
Média de idade =  39.666666666666664
Usuário mais velho =  